# Model

Import necessary libraries.

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt

from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import train_test_split

2024-04-24 13:00:18.869172: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Read the augmented data.

In [2]:
lm = pd.read_json('new_data/lm_300wLP_anno_tr.json')

## Building the Model

Preparing the Data

In [3]:
# Create the X array.
X = []
for row in lm['landmarks_2d']:
    
    # Skip the row if it has less than 12 elements.
    if len(row) < 12:
        continue
    
    # Get the first 12 elements of the array.
    row = row[:12]
    
    # Flatten the array.
    row_ = np.array(row).reshape(12, 68 * 2)
    X.append(row_)

# Create the y array.
y = []
for row in lm['landmarks']:
    
    # Skip the row if it has less than 12 elements.
    if len(row) < 12:
        continue
    
    # Get only the first element.
    row = row[0]
    
    # Flatten the array.
    row_ = np.array(row).reshape(68 * 3)
    y.append(row_)

# Convert the lists to numpy arrays.
X = np.array(X)
y = np.array(y)

In [4]:
# Train-test split
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.2, random_state=42)

In [18]:
class LandmarkModel(models.Model):
    def __init__(self, in_, out_):
        super(LandmarkModel, self).__init__()
        self.lstm1 = layers.LSTM(64, return_sequences=True, input_shape=in_)
        self.bn1 = layers.BatchNormalization()  # Batch normalization after first LSTM
        self.dropout1 = layers.Dropout(0.3)  # Dropout after batch normalization

        self.lstm2 = layers.LSTM(128, return_sequences=False)
        self.bn2 = layers.BatchNormalization()  # Batch normalization after second LSTM
        self.dropout2 = layers.Dropout(0.3)  # Dropout after batch normalization

        self.dense = layers.Dense(out_, activation='linear')

    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.bn1(x)
        x = self.dropout1(x)

        x = self.lstm2(x)
        x = self.bn2(x)
        x = self.dropout2(x)

        return self.dense(x)

In [19]:
# Get the input and output shapes.
in_ = X[0].shape
out_ = y[0].shape[0]

# Print the shapes.
print(f'{X[0].shape=}, {y[0].shape=}')

# Create the model.
model = LandmarkModel(in_, out_)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae', 'mse'])

X[0].shape=(12, 136), y[0].shape=(204,)


In [20]:
# Train the model.
history = model.fit(X_train, y_train, epochs=120, batch_size=32, validation_split=0.2)

Epoch 1/120


75/75 [==============================] - 11s 42ms/step - loss: 47651.0625 - mae: 184.4454 - mse: 47651.0625 - val_loss: 47742.2500 - val_mae: 184.7350 - val_mse: 47742.2500
Epoch 2/120
75/75 [==============================] - 1s 19ms/step - loss: 47206.5078 - mae: 183.3980 - mse: 47206.5078 - val_loss: 46675.9883 - val_mae: 182.2025 - val_mse: 46675.9883
Epoch 3/120
75/75 [==============================] - 1s 19ms/step - loss: 45666.2188 - mae: 179.6288 - mse: 45666.2188 - val_loss: 44153.3164 - val_mae: 175.9357 - val_mse: 44153.3164
Epoch 4/120
75/75 [==============================] - 1s 18ms/step - loss: 42612.3281 - mae: 171.9545 - mse: 42612.3281 - val_loss: 40755.3359 - val_mae: 167.2959 - val_mse: 40755.3359
Epoch 5/120
75/75 [==============================] - 1s 19ms/step - loss: 38458.5781 - mae: 161.4511 - mse: 38458.5781 - val_loss: 36409.6289 - val_mae: 156.2797 - val_mse: 36409.6289
Epoch 6/120
75/75 [==============================] - 1s 18ms/step - loss: 33549.8203 - mae:

In [21]:
# Evaluate the model.
loss = model.evaluate(X_test, y_test)

24/24 [==============================] - 0s 11ms/step - loss: 235.6573 - mae: 11.3266 - mse: 235.6573


In [22]:
# Get the current timestamp.
now = dt.now().strftime('%Y-%m-%d_%H-%M-%S')

# Save the model.
model.save(f'models/{now}', save_format='tf')

INFO:tensorflow:Assets written to: models/2024-04-24_13-09-38/assets


INFO:tensorflow:Assets written to: models/2024-04-24_13-09-38/assets
